In [ ]:
from flask import Flask, Response, render_template, send_file, jsonify, request
import cv2
import math
import mediapipe as mp
from ultralytics import YOLO
from gtts import gTTS
import io

# Initialize Flask app
app = Flask(__name__)

# Load YOLO model
model_yolo = YOLO(r"C:\Users\srika\Downloads\signlanguage.pt")

# MediaPipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2)
mp_drawing = mp.solutions.drawing_utils

# Define class names
classNames = ["hello", "I love you", "please", "yes", "how", "thanks", "what", "name", "you"]

# Global variable to store the current detected sign
current_sign = ""

def generate_frames():
    global current_sign
    cap = cv2.VideoCapture(0)  # Start webcam
    cap.set(3, 640)  # Set width
    cap.set(4, 640)  # Set height

    while True:
        success, img = cap.read()
        if not success:
            break

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        hand_results = hands.process(img_rgb)  # Process hand detection

        detected_sign = ""  # Local variable for detected sign

        # Use YOLO for sign detection
        results = model_yolo(img, stream=True)

        # Process YOLO results
        for r in results:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

                confidence = math.ceil((box.conf[0] * 100)) / 100
                cls = int(box.cls[0])
                detected_text = classNames[cls]

                if confidence >= 0.40:
                    detected_sign = detected_text
                    org = [x1, y1]
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    fontScale = 2
                    color = (0, 255, 0)
                    thickness = 2
                    cv2.putText(img, detected_text, org, font, fontScale, color, thickness)

        # Update the global variable with the detected sign only if no sign has been set
        if detected_sign and not current_sign:
            current_sign = detected_sign

        # Draw hand landmarks using MediaPipe
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Convert image to JPEG format
        ret, buffer = cv2.imencode('.jpg', img)
        if not ret:
            continue

        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

    cap.release()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/audio/<sign>')
def audio(sign):
    if sign in classNames:
        tts = gTTS(text=sign, lang='en')
        audio_stream = io.BytesIO()
        tts.write_to_fp(audio_stream)
        audio_stream.seek(0)
        return send_file(audio_stream, mimetype='audio/mpeg', as_attachment=False, download_name=f"{sign}.mp3")
    return "Audio not found", 404

@app.route('/clear_sign', methods=['POST'])
def clear_sign():
    global current_sign
    current_sign = ""  # Clear the current sign
    return jsonify(success=True)

@app.route('/current_sign')
def get_current_sign():
    # Return the current detected sign as JSON
    return jsonify(sign=current_sign)

if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2024 02:01:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2024 02:01:23] "GET /current_sign HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2024 02:01:24] "GET /current_sign HTTP/1.1" 200 -


127.0.0.1 - - [29/Sep/2024 02:01:25] "GET /current_sign HTTP/1.1" 200 -


0: 480x640 (no detections), 328.0ms
Speed: 4.0ms preprocess, 328.0ms inference, 6.0ms postprocess per image at shape (1, 3, 480, 640)


127.0.0.1 - - [29/Sep/2024 02:01:25] "GET /video_feed HTTP/1.1" 200 -


...

0: 480x640 (no detections), 291.6ms
Speed: 2.0ms preprocess, 291.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 288.6ms
Speed: 2.0ms preprocess, 288.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)



127.0.0.1 - - [29/Sep/2024 02:01:51] "GET /current_sign HTTP/1.1" 200 -


0: 480x640 (no detections), 300.6ms
Speed: 2.0ms preprocess, 300.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 298.6ms
Speed: 2.0ms preprocess, 298.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 296.6ms
Speed: 1.0ms preprocess, 296.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)



127.0.0.1 - - [29/Sep/2024 02:01:52] "GET /current_sign HTTP/1.1" 200 -


0: 480x640 (no detections), 299.6ms
Speed: 2.0ms preprocess, 299.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 287.6ms
Speed: 1.0ms preprocess, 287.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 322.6ms
Speed: 2.0ms preprocess, 322.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 287.6ms
Speed: 2.0ms preprocess, 287.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
